In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora

import spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/MarlonFu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/MarlonFu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv("data/choiceboard_data.csv")
df.head()

,Subj #,Unnamed: 1,Win 1_2,Win 3_4,Win 5_6,Win 7_8,Win 9_10,Unnamed: 7,Spr 1_2,Spr 3_4,Spr 5_6,Spr 7-8,Spr 9-10,Unnamed: 13,CPD_Q1,CPD_Q2,CPD_Q3
0,1,NaN,Took the time to do some meal prep last night ...,I spent time before work to do an easy 20-minu...,Took a nap before work since my team this week...,Called a friend who happened to be on her lunc...,Took a walk outside to take in some of the rar...,NaN,Had a very busy weekend with my sister visitin...,Spent some time on the phone catching up with ...,Gave myself time to lie down on the couch and ...,Took a nap after class and ended up sleeping t...,Set aside and spent some time at night after s...,NaN,The class time devoted to health and wellness ...,During my gap years I gained a lot of insight ...,I believe self-care to be really important in ...
1,2,NaN,1. Select one of the activities that resonates...,One of the activities that I chose from the Ch...,One of the activities that I chose for self-ca...,"For my self care, I went on a walk and listene...",One of the activities that I chose for health/...,NaN,I decided to spend my self care time by going ...,"For my self care, I decided to find new music ...",I went on a run!,I watched TV with my dad!,I painted my nails! This is something I really...,NaN,The class time devoted to health and wellness ...,The way I decided which self-care to choose wa...,The time spent on my choice of self-care allow...
2,3,NaN,"Today, I cooked myself lunch and it was delici...",I took a walk around campus with my roommate t...,I listened to some of my favorite artists whil...,"Today, I spent my break eating good food and w...","Today, I spent some of my time watching a show...",NaN,I took the time to finish building my baby Yod...,I took the time to reach out and talk to one o...,This week I spent at least 20 minutes reading ...,This week I spent about an hour getting dinner...,"This weekend, I went shopping with my friend a...",NaN,The class time has reminded me on the importan...,To decide on what to do it would mostly depend...,The time spent on my choice of self care affec...
3,4,NaN,I worked on my Paint by Diamond piece.,I worked on more of the Paint by Diamonds.,I worked on my Paint by Diamond while listenin...,I worked on my Paint by Diamond while listenin...,I grabbed lunch with a friend.,NaN,I started a new Asian drama (tv show) called T...,I took a walk around my neighborhood while lis...,"I took a walk to my neighborhood park , sat on...",I went to go visit my high school teachers.,I went out and had lunch with my brother.,NaN,The class time that was devoted to health and ...,"For me, it depends on the weather à if it was ...",It made me more relaxed and less stressed abou...
4,5,NaN,I took a 20 minutes walk from the train statio...,I met up with my friends for coffee and it was...,I soaked in the bathtub for an hour with relax...,I walked around the neighborhood for thirty mi...,I tried to using peloton in our apartment bui...,NaN,Yesterday was my birthday and my boyfriend pla...,I am increasing my cardio work out in the apar...,I went to watch movie at ipic theatre in redmo...,I took a 30 minutes walk in my new neighborhoo...,It was a nice day today and my friend and I to...,NaN,I have learnt to listen to people without inte...,This is because I wanted to connect with mysel...,This has been a lifeline because I always felt...


In [4]:
df_cpd = df[["CPD_Q1", "CPD_Q3"]]
df_cpd.head()

,CPD_Q1,CPD_Q3
0,The class time devoted to health and wellness ...,I believe self-care to be really important in ...
1,The class time devoted to health and wellness ...,The time spent on my choice of self-care allow...
2,The class time has reminded me on the importan...,The time spent on my choice of self care affec...
3,The class time that was devoted to health and ...,It made me more relaxed and less stressed abou...
4,I have learnt to listen to people without inte...,This has been a lifeline because I always felt...


## EDA (Q1 only)

In [5]:
corpus_1 = df_cpd["CPD_Q1"].dropna()
corpus_1 = corpus_1.astype(str)
corpus_1

0      The class time devoted to health and wellness ...
1      The class time devoted to health and wellness ...
2      The class time has reminded me on the importan...
3      The class time that was devoted to health and ...
4      I have learnt to listen to people without inte...
                             ...                        
103    It’s been a wonderful change of pace to have a...
104    The mentor team discussions with my team and p...
105    The class time devoted to health, wellness, an...
106    I personally think it was a nice break, but I ...
107    I actively disliked most of the content relate...
Name: CPD_Q1, Length: 105, dtype: object

In [6]:
len(corpus_1)

105

In [7]:
#adding clearly neutral stopwords that are specific to this program or activity
stopwords = nltk.corpus.stopwords.words('english')

stopwords.append('pharmacy')
stopwords.append('pharmacist')
stopwords.append('pharmacists')
stopwords.append('health')
stopwords.append('healthcare')
stopwords.append('practitioner')
stopwords.append('mental')
stopwords.append('wellness')
stopwords.append('self-care')
stopwords.append('mentor')
stopwords.append('WIP')
stopwords.append('class')
stopwords.append('school')
stopwords.append('peers')
stopwords.append('classmates')
stopwords.append('quarter')
stopwords.append('spring')
stopwords.append('winter')
stopwords.append('fall')

stopwords = list(stopwords)

In [8]:
sample = pd.DataFrame(corpus_1.sample(75)).reset_index()[['CPD_Q1']]
sample

,CPD_Q1
0,I found the time devoted to health and wellnes...
1,It has helped me a lot because it gives me tim...
2,I appreciate that during the Wednesday asynchr...
3,I really appreciated the in class time dedicat...
4,I have been busier over the past 3 quarters. I...
...,...
70,It’s been helpful to have open conversations w...
71,I think my favorite parts have been the mentor...
72,These past quarters have really helped me take...
73,This class time is such a mental rest for me. ...


In [9]:
sample['CPD_Q1'] = sample['CPD_Q1'].map(lambda x: re.sub('[,\.!?]', '', x))
sample['CPD_Q1'] = sample['CPD_Q1'].map(lambda x: x.lower())
sample.head()

,CPD_Q1
0,i found the time devoted to health and wellnes...
1,it has helped me a lot because it gives me tim...
2,i appreciate that during the wednesday asynchr...
3,i really appreciated the in class time dedicat...
4,i have been busier over the past 3 quarters i ...


In [10]:
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = sample.CPD_Q1.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['found', 'the', 'time', 'devoted', 'to', 'health', 'and', 'wellness', 'rewarding', 'we', 'tend', 'to', 'focus', 'so', 'much', 'on', 'learning', 'drugs', 'disease', 'states', 'that', 'we', 'forget', 'patients', 'who', 'lie', 'in', 'the', 'center', 'of']


In [11]:
#bigrams and trigrams
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [12]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['find', 'time', 'devoted', 'rewarding', 'tend', 'focus', 'much', 'learn', 'drug', 'disease', 'state', 'forget', 'patient', 'lie', 'center', 'skill', 'compassionate', 'communication', 'important', 'interact', 'people', 'career', 'need', 'learn', 'practice', 'effectively', 'interact', 'competent', 'self', 'care']


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 2), (29, 1)]


In [21]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=8, #baseline
                                       random_state=42,
                                       chunksize=10, #of docs processed
                                       passes=100, #epochs
                                       per_word_topics=True)

In [18]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.029*"time" + 0.018*"feel" + 0.017*"team" + 0.015*"session" + '
  '0.015*"think" + 0.014*"discussion" + 0.012*"devote" + 0.012*"get" + '
  '0.012*"care" + 0.011*"important"'),
 (1,
  '0.052*"time" + 0.024*"care" + 0.020*"self" + 0.019*"feel" + 0.015*"take" + '
  '0.014*"really" + 0.013*"also" + 0.012*"help" + 0.012*"focus" + 0.012*"get"'),
 (2,
  '0.025*"think" + 0.024*"also" + 0.024*"time" + 0.017*"really" + 0.017*"well" '
  '+ 0.014*"break" + 0.012*"make" + 0.012*"discussion" + 0.012*"help" + '
  '0.011*"appreciate"'),
 (3,
  '0.064*"time" + 0.031*"take" + 0.026*"care" + 0.020*"help" + 0.020*"feel" + '
  '0.019*"think" + 0.016*"find" + 0.013*"way" + 0.013*"day" + 0.012*"self"'),
 (4,
  '0.024*"time" + 0.021*"allow" + 0.014*"provide" + 0.014*"team" + '
  '0.011*"stress" + 0.011*"life" + 0.011*"appreciate" + 0.011*"work" + '
  '0.011*"space" + 0.011*"opportunity"'),
 (5,
  '0.015*"reflect" + 0.014*"past" + 0.010*"time" + 0.010*"realize" + '
  '0.010*"enjoy" + 0.010*"happy" + 0

In [19]:
from gensim.models import CoherenceModel

## BASELINE -------
#better to have higher coherence score

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

# Score < 0.2: Low coherence, topics may not be interpretable.
# 0.2 <= Score < 0.4: Moderate coherence, topics are somewhat interpretable.
# Score >= 0.4: Good coherence, topics are likely to be interpretable.

Coherence Score:  0.351720002940574


In [20]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=10,
                                           passes=100,
                                           alpha=a,
                                           eta=b)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [ ]:
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))

    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)

                    pbar.update(1)
    model_results = pd.DataFrame(model_results).to_csv('./lda_tuning_results.csv', index=False)
    pbar.close()

 77%|███████████████████████████████▌         | 416/540 [4:45:34<18:02,  8.73s/it]

In [ ]:
model_results
model_results[model_results["Coherence"] == max(model_results["Coherence"])]

""
